## Stuff Documentchain Text Summarization

In [10]:
import os 
from dotenv import load_dotenv
load_dotenv()
from langchain.schema import(
    AIMessage,
    HumanMessage,SystemMessage
)
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [18]:
from langchain_groq import ChatGroq
api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(groq_api_key=api_key, model = "llama-3.1-8b-instant") 


In [27]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()
print(docs)

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [24]:
template = """
        Write a concise and short summary of the follwing speech,
        Speech: {text}

"""
prompt = PromptTemplate(input_variable = ['text'], template=template)

In [25]:
from langchain.chains.summarize import load_summarize_chain


In [28]:
chain = load_summarize_chain(llm, chain_type = 'stuff', prompt=prompt, verbose=True)
output_summary = chain.run(docs)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

        Write a concise and short summary of the follwing speech,
        Speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as throug

## Map reduce to sumarize Large documents

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [31]:
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [32]:
final_documents = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [33]:
len(final_documents)

13

In [36]:
chunk_prompt = """
Please summarize the below speech:
Speech: {text},
Summary:
"""
map_prompt_template = PromptTemplate(input_variables = ['text'],
                                     template=chunk_prompt)


In [38]:
final_prompt = '''
Provide the final summary of the enitre speech with these important points.
Add a motivational title and start a precise summary with an intriduction and summary in number points of the speech.
Speech: {text}
'''
final_prompt_template=PromptTemplate(input_varibles=['text'], template=final_prompt)
final_prompt_template


PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of the enitre speech with these important points.\nAdd a motivational title and start a precise summary with an intriduction and summary in number points of the speech.\nSpeech: {text}\n')

In [39]:
summary_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)

output = summary_chain.run(final_documents)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have m

d:\Udemy Gen AI\Langchain Projects\venvlc\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sharm\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Token indices sequence length is longer than the specified maximum sequence length for this model (2322 > 1024). Running th



> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the enitre speech with these important points.
Add a motivational title and start a precise summary with an intriduction and summary in number points of the speech.
Speech: In his departing speech as the President of India, Dr. A.P.J. Abdul Kalam reflects on his five-year tenure and expresses gratitude to the people of India. He shares 10 key messages that he has learned during his time in office, which are:

1. Accelerate development through the aspirations of the youth.
2. Empower villages.
3. Mobilize rural core competence for competitiveness.
4. Focus on agriculture from seed to food to drive growth.
5. Overcome challenges and succeed.
6. Address problems through partnerships.
7. Demonstrate courage in the face of calamities.
8. Foster connectivity for societal transformation.
9. Defend the nation with pride.
10. Harness the energy of the youth to achieve a Developed India by 2020.

He also sh

## Refine chain sumarization

In [40]:
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True
)
chain.run(final_documents)



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have man

'Refined Summary:\n\nFormer Indian President A.P.J. Abdul Kalam, in his departing speech, shares 10 key messages from his five-year tenure, emphasizing the importance of youth empowerment, rural development, and nation-building. These messages include:\n\n1. Accelerating development led by youth aspirations\n2. Empowering villages to drive their own development\n3. Mobilizing rural core competence for competitiveness\n4. Focusing on agriculture (seed to food) for a self-sufficient nation\n5. Defeating problems and succeeding through determination and hard work\n6. Overcoming challenges through partnerships and collaboration\n7. Showing courage in the face of calamities and adversity\n8. Fostering connectivity for societal transformation and economic growth\n9. Defending the nation and its people\n10. Launching a youth movement to achieve India\'s development goals by 2020\n\nKalam\'s vision is inspired by a young girl, Anukriti, who asked why India cannot become a developed nation befo